<a href="https://colab.research.google.com/github/saykim/006958/blob/master/minibatch_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [4]:
boston = datasets.load_boston()

x = boston.data
y = boston.target

In [5]:
torch.manual_seed(1)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.2, random_state=42)

In [7]:
sc = StandardScaler()

In [8]:
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [9]:
x_train = torch.FloatTensor(x_train).to(device) #GPU 계산할때 잊지 말것!!
y_train = torch.FloatTensor(y_train).to(device) #GPU 계산할때 잊지 말것!!
x_test = torch.FloatTensor(x_test).to(device) #GPU 계산할때 잊지 말것!!
y_test = torch.FloatTensor(y_test).to(device) #GPU 계산할때 잊지 말것!!

In [63]:
y_train = y_train.view(-1,1)
y_test = y_test.view(-1,1)

In [11]:
x_train

tensor([[ 1.2877, -0.5003,  1.0332,  ...,  0.8453, -0.0743,  1.7535],
        [-0.3364, -0.5003, -0.4132,  ...,  1.2047,  0.4302, -0.5615],
        [-0.4033,  1.0133, -0.7152,  ..., -0.6372,  0.0653, -0.6516],
        ...,
        [-0.4055,  2.9593, -1.3034,  ..., -0.5923,  0.3790, -0.9107],
        [ 0.8519, -0.5003,  1.0332,  ...,  0.8453, -2.6946,  1.5226],
        [-0.3814, -0.5003, -0.3522,  ...,  1.1598, -3.1216, -0.2573]],
       device='cuda:0')

In [64]:
#dataset
dataset = TensorDataset(x_train, y_train)
dataset

In [65]:
type(dataset)

torch.utils.data.dataset.TensorDataset

In [66]:
#dataloader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [67]:
class LR_batch_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(13,1)

    def forward(self,x):
        return self.linear(x) 


In [78]:
model = LR_batch_model().to(device) #GPU 계산할때 잊지 말것!!
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [79]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.2753,  0.0393, -0.1892,  0.1874,  0.0469, -0.0879, -0.1967, -0.1174,
          0.0164, -0.1025, -0.2176, -0.2751, -0.0039]], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([0.2683], device='cuda:0', requires_grad=True)]


In [80]:
n_epochs = 10000

for epoch in range(n_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        pred = model(x_train)

        cost = F.mse_loss(pred, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, batch_idx+1, len(dataloader),
            cost.item()))


Epoch    0/10000 Batch 1/1 Cost: 365.474182
Epoch 1000/10000 Batch 1/1 Cost: 221.506393
Epoch 2000/10000 Batch 1/1 Cost: 125.731323
Epoch 3000/10000 Batch 1/1 Cost: 63.473728
Epoch 4000/10000 Batch 1/1 Cost: 26.814697
Epoch 5000/10000 Batch 1/1 Cost: 8.910526
Epoch 6000/10000 Batch 1/1 Cost: 2.682580
Epoch 7000/10000 Batch 1/1 Cost: 1.390353
Epoch 8000/10000 Batch 1/1 Cost: 1.040653
Epoch 9000/10000 Batch 1/1 Cost: 0.709247
Epoch 10000/10000 Batch 1/1 Cost: 0.397974


In [81]:
pred_y_test = model(x_test)

In [84]:
F.mse_loss(pred_y_test, y_test)

tensor(589.3848, device='cuda:0', grad_fn=<MseLossBackward>)